### Add incore/ergo name space to dc:title

In [1]:
from pymongo import MongoClient

In [2]:
mongo_username = "root"
mongo_password = "ask developer for password"

host = "localhost"
port = "27019" # dev
# port = "27020" # prod
# port = "27017" # local

client = MongoClient('mongodb://%s:%s@%s:%s' % (mongo_username, mongo_password, host, port))

In [14]:
incore_title_name = []
for document in client['semanticsdb']['incore'].find():
    title = document["dc:title"].replace("ergo-", "").replace("_1.0", "").replace("incore-", "")
    if title not in incore_title_name:
        incore_title_name.append(title)

In [15]:
incore_title_name

['electricFacilityRestorationCurve',
 'electricFacilityRestorationMapping',
 'buildingTornadoDamage',
 'censusBlockgroupTable',
 'cgeBuildingToSector',
 'cgeResults',
 'electricPowerLineFragilities',
 'electricPowerLineFragilityMapping',
 'epaNetWaterFacility',
 'epaNetWaterPipeline',
 'epaNetWaterUsagePattern',
 'landcoverRaster',
 'mraBuildingData',
 'mraOLSParameters',
 'mraSummaryTable',
 'populationVulnerability',
 'populationVulnerabilityDdi',
 'populationVulnerabilitySumStats',
 'populationVulnerabilitySumStatsDdi',
 'potableWaterFacility',
 'potableWaterPipeline',
 'roadwayDamage',
 'roadwayDamageRatios',
 'roadwayFragilities',
 'roadwayFragilityMapping',
 'roadwayLink',
 'roadwayRestoration',
 'roadwayRestorationCurve',
 'roadwayRestorationMapping',
 'roadwayRestorationResult',
 'tornadoEPNDamage',
 'tornadohazard',
 'tornadoWindRaster',
 'wildfireDamageRaster',
 'BuildingDamageCapacityCurves',
 'BuildingRepairCostRatio',
 'BuildingStandardDeviationDamageFactors',
 'DirectLoss

In [17]:
ergo_title_name = []
for document in client['semanticsdb']['ergo'].find():
    title = document["dc:title"].replace("ergo-", "").replace("_1.0", "").replace("incore-", "")
    if title not in incore_title_name:
        ergo_title_name.append(title)

In [16]:
ergo_title_name

['defaultColumns-guid',
 'defaultColumns-structuralDamage',
 'annualGrossSalesOrProduction',
 'attenuationValidationResult',
 'blockGroupPopulationDislocation',
 'bridgeDamage',
 'bridgeDamageRatios',
 'bridgeFragilities',
 'bridgeFragilityMapping',
 'bridgeFunctionality',
 'bridgeFunctionalityResults',
 'bridgeOnLink',
 'bridgeRepairCostData',
 'bridgeRepairCostResults',
 'bridgeRetrofitCost',
 'bridgeRetrofitCostBenefitResult',
 'bridgeRetrofitCostEstimationResult',
 'bridgeRetrofitCostResults',
 'bridges',
 'buildingASDamageRatios',
 'buildingCollapseRateTable',
 'buildingContentDamageRatios',
 'buildingCostBenefitResult',
 'buildingDamageRatios',
 'buildingDamageVer4',
 'buildingDecisionAttributeResults',
 'buildingDiscountRateMapping',
 'buildingDisruptionCost',
 'buildingDSDamageRatios',
 'buildingEconomicLoss',
 'buildingEconomicLossVer4',
 'buildingEquivalentCostAnalysis',
 'buildingExpectedDeaths',
 'buildingExpectedInjuries',
 'buildingFireIgnition',
 'buildingFireIgnitionOcc

In [19]:
for document in client['semanticsdb']['Type'].find():
    # 
    if document["dc:title"] in incore_title_name:
        document["dc:title"] = document["dc:title"][0].lower() + document["dc:title"][1:]
        document["dc:title"] = "incore:" + document["dc:title"]
    elif document["dc:title"] in ergo_title_name:
        document["dc:title"] = document["dc:title"][0].lower() + document["dc:title"][1:]
        document["dc:title"] = "ergo:" + document["dc:title"]
    else:
        document["dc:title"] = document["dc:title"][0].lower() + document["dc:title"][1:]
        print(document["dc:title"])
    client['semanticsdb']['Type'].replace_one({'_id':document['_id']}, document)

gasUtilityNetworkDamage
waterUtilityNetworkDamage
ddiCensusBlockgroupXML
waterFacility
gis-defaultset
powerUtilityNetworkDamage
cgeShock
mraSolutions
epaNetInputFile
waterUtilityNetwork
gis-regionOfInterest
gis-anonymousTextFiles
scenario
tempHousingMetricsResult
temporaryHousingAnalyzedData
odTextFiles
tempHousingMAU
powerUtilityNetwork
gasUtilityNetwork
substationFragilityMapping


In [34]:
for document in client['semanticsdb']['Type'].find():
    document["url"] = document["dc:title"]
    client['semanticsdb']['Type'].replace_one({'_id':document['_id']}, document)

In [37]:
ergo_doc = client['spacedb']['Space'].find_one({"metadata.name": "ergo"})
incore_doc = client['spacedb']['Space'].find_one({"metadata.name": "incore"})

In [36]:
# ingest all the ids accordingly to different space collection
for document in client['semanticsdb']['Type'].find():
    if document["url"].startswith("ergo:") and document["_id"] not in ergo_doc["members"]:
        ergo_doc["members"].append(str(document["_id"]))

client['spacedb']['Space'].replace_one({'_id':ergo_doc['_id']}, ergo_doc)

In [38]:
# ingest all the ids accordingly to different space collection
for document in client['semanticsdb']['Type'].find():
    if document["url"].startswith("incore:") and document["_id"] not in incore_doc["members"]:
        incore_doc["members"].append(str(document["_id"]))

client['spacedb']['Space'].replace_one({'_id':incore_doc['_id']}, incore_doc)